In [1]:
import numpy as np
import pandas as pd
import re
import glob
import os 
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV 

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
path_train="scores/stack_train/"
path_test="scores/stack_test/"

In [7]:
dict_train = {} 
dict_test = {}
train_files = [i for i in os.listdir(path_train)]
test_files = [i for i in os.listdir(path_test)]
for f in train_files:
    dict_train[re.sub(".csv","",f)] = pd.read_csv(path_train+f)
for f in test_files:
    dict_test[re.sub(".csv","",f)] = pd.read_csv(path_test+f)

In [10]:
for files in glob.glob(path_test+"*.csv"):
    print (files) 

scores/stack_test\nowdw_neural_net.csv
scores/stack_test\nowdw_XGB_medium.csv
scores/stack_test\nowdw_XGB_small.csv


In [5]:
stack_train=dict_train["XGB_small"].merge(dict_train["XGB_medium"], on="ID").merge(dict_train["XGB_tall"],on="ID").merge(dict_train["LGBM_small"],on="ID").merge(dict_train["LGBM_mid"],on="ID").merge(dict_train["LGBM_tall"],on="ID")
#a tester 

In [8]:
stack_test=dict_test["XGB_small"].merge(dict_test["XGB_medium"], on="ID").merge(dict_test["XGB_tall"],on="ID").merge(dict_test["LGBM_small"],on="ID").merge(dict_test["LGBM_mid"],on="ID").merge(dict_test["LGBM_tall"],on="ID")

In [9]:
#quand le merging est fini : table unique, on peut monter un meta_model

In [10]:
y=pd.read_csv("bases/training_variants").Class.values - 1

In [11]:
ID_test=stack_test["ID"]

In [12]:
X_train=np.array(stack_train.drop("ID",axis=1))

In [13]:
X_test=np.array(stack_test.drop("ID",axis=1))

In [16]:
param_test1 = {
 'max_depth':[1,3,5,7,9],
 'min_child_weight':[1,3,5,7,9]
}
param_test1_bis={
    'max_depth':[1,2,3,4,5,7,9],
 'min_child_weight':[1,3,5,7]
}
param_test2 = {
 'gamma':[i for i in range(1,10)]
}
param_test3 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
param_test3_bis = {
 'subsample':[i/100.0 for i in range(75,90,5)],
 'colsample_bytree':[i/100.0 for i in range(75,90,5)]
}
param_test4 = {
 'reg_alpha':[1,2,3,4,5,10,50, 100]
}
param_test4_bis = {
 'reg_alpha':[i for i in np.arange(94,96,.2)]
}
param_test5 = {
 "learning_rate" : [i for i in np.arange(0.05,0.15,0.025)]
}
param_test6 = {
    "n_estimators" : [i for i in range(50, 300, 50)]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26),param_grid = param_test1,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
#gsearch1.fit(X_train,y)
#gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [17]:
gsearch1_bis=GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,gamma=6,n_estimators=140),param_grid = param_test1_bis,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch1_bis.fit(X_train,y)
gsearch1_bis.grid_scores_, gsearch1_bis.best_params_, gsearch1_bis.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.96434, std: 0.08249, params: {'max_depth': 1, 'min_child_weight': 1},
  mean: -0.96437, std: 0.08244, params: {'max_depth': 1, 'min_child_weight': 3},
  mean: -0.96449, std: 0.08214, params: {'max_depth': 1, 'min_child_weight': 5},
  mean: -0.96511, std: 0.08110, params: {'max_depth': 1, 'min_child_weight': 7},
  mean: -0.96922, std: 0.08693, params: {'max_depth': 2, 'min_child_weight': 1},
  mean: -0.96850, std: 0.08794, params: {'max_depth': 2, 'min_child_weight': 3},
  mean: -0.96888, std: 0.08690, params: {'max_depth': 2, 'min_child_weight': 5},
  mean: -0.96853, std: 0.08545, params: {'max_depth': 2, 'min_child_weight': 7},
  mean: -0.97494, std: 0.08648, params: {'max_depth': 3, 'min_child_weight': 1},
  mean: -0.97596, std: 0.08686, params: {'max_depth': 3, 'min_child_weight': 3},
  mean: -0.97495, std: 0.08801, params: {'max_depth': 3, 'min_child_weight': 5},
  mean: -0.97384, std: 0.08715, params: {'max_depth': 3, 'min_child_weight': 7},
  mean: -0.98032, std: 0.087

In [18]:
gsearch2 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=1,min_child_weight=3,gamma=3),param_grid = param_test2,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.96230, std: 0.09001, params: {'gamma': 1},
  mean: -0.96149, std: 0.09003, params: {'gamma': 2},
  mean: -0.96216, std: 0.08906, params: {'gamma': 3},
  mean: -0.96120, std: 0.08663, params: {'gamma': 4},
  mean: -0.96177, std: 0.08450, params: {'gamma': 5},
  mean: -0.96436, std: 0.08230, params: {'gamma': 6},
  mean: -0.96746, std: 0.08049, params: {'gamma': 7},
  mean: -0.97040, std: 0.07928, params: {'gamma': 8},
  mean: -0.97408, std: 0.07741, params: {'gamma': 9}],
 {'gamma': 4},
 -0.96119845675931237)

In [19]:
gsearch3 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=1,min_child_weight=3,gamma=3),param_grid = param_test3,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.96334, std: 0.09230, params: {'colsample_bytree': 0.6, 'subsample': 0.6},
  mean: -0.96331, std: 0.09117, params: {'colsample_bytree': 0.6, 'subsample': 0.7},
  mean: -0.96213, std: 0.09044, params: {'colsample_bytree': 0.6, 'subsample': 0.8},
  mean: -0.96064, std: 0.09195, params: {'colsample_bytree': 0.6, 'subsample': 0.9},
  mean: -0.96325, std: 0.09033, params: {'colsample_bytree': 0.7, 'subsample': 0.6},
  mean: -0.96336, std: 0.09111, params: {'colsample_bytree': 0.7, 'subsample': 0.7},
  mean: -0.96068, std: 0.09013, params: {'colsample_bytree': 0.7, 'subsample': 0.8},
  mean: -0.95979, std: 0.09066, params: {'colsample_bytree': 0.7, 'subsample': 0.9},
  mean: -0.96408, std: 0.09154, params: {'colsample_bytree': 0.8, 'subsample': 0.6},
  mean: -0.96357, std: 0.09087, params: {'colsample_bytree': 0.8, 'subsample': 0.7},
  mean: -0.96219, std: 0.08942, params: {'colsample_bytree': 0.8, 'subsample': 0.8},
  mean: -0.96141, std: 0.08988, params: {'colsample_bytree': 0.8,

In [20]:
gsearch4 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=1,min_child_weight=3,gamma=3,colsample_bytree=0.8,subsample=0.9),param_grid = param_test4,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch4.fit(X_train,y)
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.96166, std: 0.08835, params: {'reg_alpha': 1},
  mean: -0.96176, std: 0.08689, params: {'reg_alpha': 2},
  mean: -0.96339, std: 0.08452, params: {'reg_alpha': 3},
  mean: -0.96428, std: 0.08255, params: {'reg_alpha': 4},
  mean: -0.96678, std: 0.08180, params: {'reg_alpha': 5},
  mean: -0.97901, std: 0.07620, params: {'reg_alpha': 10},
  mean: -1.12064, std: 0.05085, params: {'reg_alpha': 50},
  mean: -1.30463, std: 0.02686, params: {'reg_alpha': 100}],
 {'reg_alpha': 1},
 -0.96165883110155259)

In [21]:
gsearch5 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=1,min_child_weight=3,gamma=3,colsample_bytree=0.8,subsample=0.9,learning_rate=0.075),param_grid = param_test5,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_train,y)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.96434, std: 0.08111, params: {'learning_rate': 0.050000000000000003},
  mean: -0.95822, std: 0.08702, params: {'learning_rate': 0.075000000000000011},
  mean: -0.96141, std: 0.08988, params: {'learning_rate': 0.10000000000000002},
  mean: -0.96533, std: 0.09216, params: {'learning_rate': 0.12500000000000003}],
 {'learning_rate': 0.075000000000000011},
 -0.95821688086551138)

In [22]:
gsearch6 = GridSearchCV(estimator = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=1,min_child_weight=3,gamma=3,colsample_bytree=0.8,subsample=0.9,learning_rate=0.075),param_grid = param_test6,scoring="neg_log_loss",n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_train,y)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

C:\Users\Franck\Documents\Continuum\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.98883, std: 0.07350, params: {'n_estimators': 50},
  mean: -0.95822, std: 0.08702, params: {'n_estimators': 100},
  mean: -0.96291, std: 0.09134, params: {'n_estimators': 150},
  mean: -0.96588, std: 0.09135, params: {'n_estimators': 200},
  mean: -0.96727, std: 0.09281, params: {'n_estimators': 250}],
 {'n_estimators': 100},
 -0.95821688086551138)

In [23]:
clf = XGBClassifier(objective="multi:softprob",
seed=26,max_depth=1,min_child_weight=3,gamma=3,colsample_bytree=0.8,subsample=0.9,learning_rate=0.075)
                     
clf.fit(X_train,y)
y_test=clf.predict_proba(X_test)

classes = "class1,class2,class3,class4,class5,class6,class7,class8,class9".split(',')
sub = pd.DataFrame(y_test, columns=classes)
sub['ID'] = ID_test
sub.to_csv("scores/stacking_finish.csv",index=False)